In [ ]:
# Install necessary packages
!pip install scorecard-ai
!pip install openai

In [ ]:
# Import required libraries
import os
import pandas as pd
import openai
from scorecard.client import Scorecard

# Set your API keys
SCORECARD_API_KEY = "YOUR_SCORECARD_API_KEY"  # Replace with your Scorecard API key
OPENAI_API_KEY = "YOUR_OPENAI_API_KEY"        # Replace with your OpenAI API key

# Initialize the Scorecard client
client = Scorecard(api_key=SCORECARD_API_KEY)

# Set OpenAI API key
openai.api_key = OPENAI_API_KEY

In [ ]:
# Read the dataset
df = pd.read_csv('dataset.csv')  # Ensure 'dataset.csv' is in your working directory

In [ ]:
# Create a Testset in Scorecard
testset = client.testset.create(
    name="Demo Testset",
    description="Testset created from CSV file",
)
testset_id = testset.id

# Add Testcases to the Testset
for index, row in df.iterrows():
    client.testcase.create(
        testset_id=testset_id,
        user_query=row['user_query'],
        ideal_response=row['ideal']
    )

print("Testset created with ID:", testset_id)
print(f"Visit the Scorecard UI to view your Testset: https://app.getscorecard.ai/view-dataset/{testset_id}")

In [ ]:
# Define your model function
def get_model_response(user_query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_query},
        ]
    )
    return response['choices'][0]['message']['content']

In [ ]:
# Create a Run
run = client.run.create(testset_id=testset_id)
run_id = run.id
client.run.update_status(run_id=run_id, status="running_execution")

# Execute Testcases and collect responses
testcases = client.testset.get_testcases(testset_id=testset_id).results

for testcase in testcases:
    user_query = testcase.user_query
    ideal_response = testcase.ideal_response

    model_response = get_model_response(user_query)

    client.testrecord.create(
        run_id=run_id,
        testset_id=testset_id,
        testcase_id=testcase.id,
        user_query=user_query,
        response=model_response,
        ideal_response=ideal_response
    )

client.run.update_status(run_id=run_id, status="awaiting_scoring")

print("Run created with ID:", run_id)
print(f"Visit the Scorecard UI to view your Run: https://app.getscorecard.ai/view-records/{run_id}")

### Note:
Scoring needs to be initiated from the Scorecard UI.

Please go to the Scorecard UI to initiate scoring and view results.

Visit: `https://app.getscorecard.ai/view-grades/{run_id}`